# Препроцессор данных с сайта meteo.ru
В следующей ячейке в переменную path замените путь к архиву на путь к вашему архиву с данными
Если Вам нужны данные в CSV, переменная excel должна быть равна False, если нужны данные в XLSX - должна быть равна True
Затем поочередно запустите обе ячейки скрипта
Итог работы скрипта - файл, который будет сохранен в ту же папку, где находится архив

In [ ]:
path = r'C:\Users\username\Downloads\wr60664.zip'
excel = False

In [ ]:
import zipfile
import tempfile
import re
from glob import glob
import pandas as pd


temp_dir = tempfile.TemporaryDirectory()
with zipfile.ZipFile(path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir.name)
files = glob(temp_dir.name + r'\*')
fld = next(x for x in files if re.search('fld\d', x) != None)
fldf = open(fld)
fld = fldf.read()
fldf.close()
for i in range(0,50):
    fld = fld.replace('  ', ' ')
fld = re.split('\n', fld)
del fld[-1]
columns = []
for i in fld:
    col = re.split('\d', i)
    col = col[-1]
    col = col[1:]
    columns.append(col)
statlist = next(x for x in files if re.search('statlist\d', x) != None)
statf = open(statlist)
statlist = statf.read()
statf.close()
statlist
for i in range(0,50):
    statlist = statlist.replace('  ', ' ')
statlist = re.split('\n', statlist)
del statlist[-1]
statlist
stations = {}
for i in statlist:
    st = re.split(' ', i, maxsplit=1)
    stations[st[0]] = st[1]
wr = pd.read_csv(next(x for x in files if re.search('wr\d', x) != None), sep = ';', header = None, names= columns)
wr['Станция'] = wr.apply(lambda row: stations[str(int(row['Индекс ВМО']))], axis = 1)
cols = list(wr)
cols.insert(0, cols.pop(cols.index('Станция')))
wr = wr.loc[:, cols]
temp_dir.cleanup()
if excel == True:
    final_path = path.replace('.zip', '.xlsx')
    wr.to_excel(final_path)
else:
    final_path = path.replace('.zip', '.csv')
    wr.to_csv(final_path)    
wr